In [3]:
# Load libraries
import numpy as np
import pandas as pd
import io

#  Mock Midterm Exercise

In this exercise you will have to implement code in the sections inside *Fill with Your Code* (*Load Data*, *Data Preprocessing* and *Create Model*) create a model to predict the column *exitus* in the dataset given. The dataset is already split into train, validation, and test subsets. To see to which subset belong each observation, you need to check the *dataset* column.

The code that is already written in this notebook **CANNOT BE CHANGED**. You can only add code in the *Fill with Your Code* section.

You must achieve in the last cell of this notebook an **AUC over test of at least 0.93**.

# Fill with Your Code

### [1] Load Data

In [4]:
from google.colab import files

In [5]:
uploaded = files.upload()

Saving dat.csv to dat.csv


In [7]:
dat = pd.read_csv(io.BytesIO(uploaded['dat.csv']), sep = ",")

In [8]:
dat

,date,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm,exitus,dataset
0,2016-01,NaN,0.408730,12596.0,21.0,0.0,NaN,151.0,Q,1.0,0,train
1,2016-01,NaN,0.306931,20973.0,22.0,NaN,NaN,99.0,Q,1.0,0,train
2,2016-01,4.0,0.278481,19611.0,19.0,NaN,NaN,87.0,NaN,1.0,0,train
3,2016-01,3.0,0.150289,13583.0,22.0,NaN,NaN,100.0,Q,NaN,0,train
4,2016-01,1.0,0.016573,18042.0,2.0,NaN,NaN,44.0,Q,1.0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...
32701,2016-12,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0,0,test
32702,2016-12,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0,0,test
32703,2016-12,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN,0,test
32704,2016-12,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0,0,test


In [9]:
dat.dtypes

date                object
severity           float64
mortality_ratio    float64
age                float64
num_proc           float64
ambulatory         float64
origin             float64
expected_length    float64
tip_grd             object
tip_adm            float64
exitus               int64
dataset             object
dtype: object

### [2]  Data Preprocessing

In [10]:
dat.drop('date', axis = 1, inplace = True)

In [11]:
dat_old = dat.copy()

### Encoding of Categorical Variables

In [12]:
categorical_vars = ['severity', 'origin', 'ambulatory', 'tip_grd', 'tip_adm']
non_categorical_vars = list(set(dat.columns) - set(categorical_vars))

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
# 3) Define "model"
ohe = OneHotEncoder(sparse_output = False)

# 4) Train "model"
ohe.fit(dat[categorical_vars][dat['dataset'] == 'train'])

# 5) "Predict"
dat_ohe = pd.DataFrame(ohe.transform(dat[categorical_vars]))

# Optional
dat_ohe.columns = ohe.get_feature_names_out()

# Combine numerical and categorical
dat = pd.concat((dat[non_categorical_vars], dat_ohe), axis=1)

In [15]:
dat

,exitus,expected_length,num_proc,dataset,mortality_ratio,age,severity_1.0,severity_2.0,severity_3.0,severity_4.0,...,ambulatory_0.0,ambulatory_1.0,ambulatory_nan,tip_grd_M,tip_grd_Q,tip_grd_nan,tip_adm_1.0,tip_adm_2.0,tip_adm_3.0,tip_adm_nan
0,0,151.0,21.0,train,0.408730,12596.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0,99.0,22.0,train,0.306931,20973.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0,87.0,19.0,train,0.278481,19611.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0,100.0,22.0,train,0.150289,13583.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0,44.0,2.0,train,0.016573,18042.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32701,0,2.0,2.0,test,0.028365,23619.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
32702,0,2.0,1.0,test,0.000606,3935.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
32703,0,2.0,4.0,test,0.040452,30163.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
32704,0,0.0,4.0,test,0.000000,29012.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


### Fill Missing Values

In [16]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32706 entries, 0 to 32705
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   exitus           32706 non-null  int64  
 1   expected_length  30833 non-null  float64
 2   num_proc         31356 non-null  float64
 3   dataset          32706 non-null  object 
 4   mortality_ratio  30438 non-null  float64
 5   age              32282 non-null  float64
 6   severity_1.0     32706 non-null  float64
 7   severity_2.0     32706 non-null  float64
 8   severity_3.0     32706 non-null  float64
 9   severity_4.0     32706 non-null  float64
 10  severity_nan     32706 non-null  float64
 11  origin_1.0       32706 non-null  float64
 12  origin_2.0       32706 non-null  float64
 13  origin_3.0       32706 non-null  float64
 14  origin_4.0       32706 non-null  float64
 15  origin_6.0       32706 non-null  float64
 16  origin_8.0       32706 non-null  float64
 17  origin_9.0  

In [20]:
# !pip install fancyimpute
from fancyimpute import IterativeImputer as MICE # pip install fancyimpute


In [21]:
numerical_vars = list(set(dat.columns) - set(['exitus', 'dataset']))

In [22]:
# 3) Define "model"
model = MICE()

# 4) Train "model"
model.fit(dat[numerical_vars][dat['dataset'] == 'train'])

# 5) "Predict"
dat[numerical_vars] = model.transform(dat[numerical_vars])

In [23]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32706 entries, 0 to 32705
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   exitus           32706 non-null  int64  
 1   expected_length  32706 non-null  float64
 2   num_proc         32706 non-null  float64
 3   dataset          32706 non-null  object 
 4   mortality_ratio  32706 non-null  float64
 5   age              32706 non-null  float64
 6   severity_1.0     32706 non-null  float64
 7   severity_2.0     32706 non-null  float64
 8   severity_3.0     32706 non-null  float64
 9   severity_4.0     32706 non-null  float64
 10  severity_nan     32706 non-null  float64
 11  origin_1.0       32706 non-null  float64
 12  origin_2.0       32706 non-null  float64
 13  origin_3.0       32706 non-null  float64
 14  origin_4.0       32706 non-null  float64
 15  origin_6.0       32706 non-null  float64
 16  origin_8.0       32706 non-null  float64
 17  origin_9.0  

### [3] Create Model

In [24]:
from sklearn.ensemble import RandomForestClassifier as model_constructor
from sklearn.metrics import roc_auc_score as metric

In [ ]:
# Random Forest
n_estimators_values = [10, 100, 1000]
max_features_values = [2, 5, 10]
max_samples_values = [100, 1000, dat[dat['dataset'] == 'train'].shape[0]]

params_grid = {'max_features': max_features_values,
              'n_estimators': n_estimators_values,
               'max_samples': max_samples_values}



In [ ]:
num_iter = 1;
grid_results = pd.DataFrame(columns = ('max_features',
                                       'n_estimators',
                                       'max_samples',
                                       'metric_train',
                                       'metric_val'))

for max_features in params_grid['max_features']:
    for n_estimators in params_grid['n_estimators']:
        for max_samples in params_grid['max_samples']:

                        # Print trace
                        print('Iteracion = ' + str(num_iter))

                        # [3] Define model
                        model = model_constructor(max_features = max_features,
                                                  n_estimators = n_estimators,
                                                  max_samples = max_samples,
                                                  random_state = 0)

                        # [4] Train model
                        model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1),
                                  dat[dat['dataset'] == 'train'].exitus.values)


                        # [5] Predict
                        pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
                        pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!

                        # [6] Compute metric
                        metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
                        metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])

                        # print error
                        print('Metric train = %.2f - Metric validation = %.2f.'
                              % (metric_train, metric_val))

                        # Save iteration results
                        grid_results.loc[num_iter]=[ max_features,
                                                    n_estimators,
                                                    max_samples,
                                                 metric_train,
                                                 metric_val]
                        num_iter += 1



In [ ]:
grid_results = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model = grid_results.iloc[0]
best_model

max_features       2.000000
n_estimators    1000.000000
max_samples     1000.000000
mape_train         0.962906
mape_val           0.936497
Name: 8, dtype: float64

In [25]:
model =  model_constructor(max_features = int(best_model['max_features']),
                                                  n_estimators = int(best_model['n_estimators']),
                                                  max_samples = int(best_model['max_samples']),
                                                  random_state = 0)

In [35]:
# Extract variable importance (tree-based)
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)
var_imp = pd.DataFrame({'var': dat.drop(['exitus', 'dataset'], axis = 1).columns, 'imp': model.feature_importances_})
var_imp.sort_values(['imp'], ascending = False, inplace = True)
var_imp

,var,imp
2,mortality_ratio,0.311951
3,age,0.247501
0,expected_length,0.159118
1,num_proc,0.129031
7,severity_4.0,0.018992
22,tip_grd_nan,0.014244
20,tip_grd_M,0.014072
16,origin_nan,0.012565
9,origin_1.0,0.012338
8,severity_nan,0.011880


In [47]:
# Permutation importance: https://scikit-learn.org/stable/modules/permutation_importance.html
from sklearn.inspection import permutation_importance
res = permutation_importance(model,
                       dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1),
                       dat[dat['dataset'] == 'val'].exitus.values)
var_imp = pd.DataFrame({'var': dat.drop(['exitus', 'dataset'], axis = 1).columns, 'imp': res['importances_mean']})
var_imp.sort_values(['imp'], ascending = False, inplace = True)
var_imp

,var,imp
0,expected_length,0.003996
2,mortality_ratio,0.002813
1,num_proc,0.000652
22,tip_grd_nan,0.000326
26,tip_adm_nan,0.000245
19,ambulatory_nan,0.000122
13,origin_6.0,0.000000
15,origin_9.0,0.000000
25,tip_adm_3.0,0.000000
11,origin_3.0,0.000000


In [48]:
dat = dat[['mortality_ratio', 'age', 'expected_length', 'exitus', 'dataset']] # This is just a fake example of how to select the most important variables.

# Evaluate Model

In [ ]:
# [4] Train model
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)


# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

In [ ]:
# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.96 - Metric val = 0.94 - Metric test = 0.93
